# VQGAN JAX Encoding for `webdataset`

This notebook shows how to pre-encode images to token sequences using JAX, VQGAN and a dataset in the [`webdataset` format](https://webdataset.github.io/webdataset/).

This example uses a small subset of YFCC100M we created for testing, but it should be easy to adapt to any other image/caption dataset in the `webdataset` format.

In [ ]:
import numpy as np
from tqdm import tqdm

import torch
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torchvision.transforms import InterpolationMode
import math

import webdataset as wds

import jax
from jax import pmap

## Dataset and Parameters

The following is the list of shards we'll process. We hardcode the length of data so that we can see nice progress bars using `tqdm`.

In [ ]:
shards = 'https://huggingface.co/datasets/dalle-mini/YFCC100M_OpenAI_subset/resolve/main/data/shard-{0000..0008}.tar'
length = 8320

If we are extra cautious or our server is unreliable, we can enable retries by providing a custom `curl` retrieval command:

In [ ]:
# Enable curl retries to try to work around temporary network / server errors.
# This shouldn't be necessary when using reliable servers.
# shards = f'pipe:curl -s --retry 5 --retry-delay 5 -L {shards} || true'

In [ ]:
from pathlib import Path

# Output directory for encoded files
encoded_output = Path.home()/'data'/'wds'/'encoded'

batch_size = 128           # Per device
num_workers = 8            # For parallel processing

In [ ]:
bs = batch_size * jax.device_count()    # You can use a smaller size while testing
batches = math.ceil(length / bs)

Image processing

In [ ]:
def center_crop(image, max_size=256):
    # Note: we allow upscaling too. We should exclude small images.    
    image = TF.resize(image, max_size, interpolation=InterpolationMode.LANCZOS)
    image = TF.center_crop(image, output_size=2 * [max_size])
    return image

preprocess_image = T.Compose([
    center_crop,
    T.ToTensor(),
    lambda t: t.permute(1, 2, 0)   # Reorder, we need dimensions last
])

Caption preparation.

Note that we receive the contents of the `json` structure, which will be replaced by the string we return.
If we want to keep other fields inside `json`, we can add `caption` as a new field.

In [ ]:
def create_caption(item):
    title = item['title_clean'].strip()
    description = item['description_clean'].strip()
    if len(title) > 0 and title[-1] not in '.!?': title += '.'
    return f'{title} {description}'

When an error occurs (a download is disconnected, an image cannot be decoded, etc) the process stops with an exception. We can use one of the exception handlers provided by the `webdataset` library, such as `wds.warn_and_continue` or `wds.ignore_and_continue` to ignore the offending entry and keep iterating.

**IMPORTANT WARNING:** Do not use error handlers to ignore exceptions until you have tested that your processing pipeline works fine. Otherwise, the process will continue trying to find a valid entry, and it will consume your whole dataset without doing any work.

We can also create our custom exception handler as demonstrated here:

In [ ]:
# UNUSED - Log exceptions to a file
def ignore_and_log(exn):
    with open('errors.txt', 'a') as f:
        f.write(f'{repr(exn)}\n')
    return True

In [ ]:
# Or simply use `wds.ignore_and_continue`
exception_handler = wds.warn_and_continue

In [ ]:
dataset = wds.WebDataset(shards,
                         length=batches,              # Hint so `len` is implemented
                         shardshuffle=False,          # Keep same order for encoded files for easier bookkeeping. Set to `True` for training.
                         handler=exception_handler,   # Ignore read errors instead of failing.
)

dataset = (dataset           
      .decode('pil')                     # decode image with PIL
#       .map_dict(jpg=preprocess_image, json=create_caption, handler=exception_handler)    # Process fields with functions defined above
      .map_dict(jpg=preprocess_image, json=create_caption)    # Process fields with functions defined above
      .to_tuple('__key__', 'jpg', 'json') # filter to keep only key (for reference), image, caption.
      .batched(bs))                      # better to batch in the dataset (but we could also do it in the dataloader) - this arg does not affect speed and we could remove it

In [ ]:
%%time
keys, images, captions = next(iter(dataset))

In [ ]:
images.shape

In [ ]:
T.ToPILImage()(images[0].permute(2, 0, 1))

### Torch DataLoader

In [ ]:
dl = torch.utils.data.DataLoader(dataset, batch_size=None, num_workers=num_workers)

## VQGAN-JAX model

In [ ]:
from vqgan_jax.modeling_flax_vqgan import VQModel

We'll use a VQGAN trained with Taming Transformers and converted to a JAX model.

In [ ]:
model = VQModel.from_pretrained("flax-community/vqgan_f16_16384")

## Encoding

Encoding is really simple using `shard` to automatically distribute "superbatches" across devices, and `pmap`. This is all it takes to create our encoding function, that will be jitted on first use.

In [ ]:
from flax.training.common_utils import shard
from functools import partial

In [ ]:
@partial(jax.pmap, axis_name="batch")
def encode(batch):
    # Not sure if we should `replicate` params, does not seem to have any effect
    _, indices = model.encode(batch)
    return indices

### Encoding loop

In [ ]:
import os
import pandas as pd

def encode_captioned_dataset(dataloader, output_dir, save_every=14):
    output_dir.mkdir(parents=True, exist_ok=True)

    # Saving strategy:
    # - Create a new file every so often to prevent excessive file seeking.
    # - Save each batch after processing.
    # - Keep the file open until we are done with it.
    file = None        
    for n, (keys, images, captions) in enumerate(tqdm(dataloader)):
        if (n % save_every == 0):
            if file is not None:
                file.close()
            split_num = n // save_every
            file = open(output_dir/f'split_{split_num:05x}.jsonl', 'w')

        images = shard(images.numpy().squeeze())
        encoded = encode(images)
        encoded = encoded.reshape(-1, encoded.shape[-1])

        encoded_as_string = list(map(lambda item: np.array2string(item, separator=',', max_line_width=50000, formatter={'int':lambda x: str(x)}), encoded))
        batch_df = pd.DataFrame.from_dict({"key": keys, "caption": captions, "encoding": encoded_as_string})
        batch_df.to_json(file, orient='records', lines=True)

Create a new file every 318 iterations. This should produce splits of ~500 MB each, when using a total batch size of 1024.

In [ ]:
save_every = 318

In [ ]:
encode_captioned_dataset(dl, encoded_output, save_every=save_every)

----